In [ ]:
!pip install datasketch

In [ ]:
import json
import time
from google.colab import drive
from datasketch import HyperLogLog

In [ ]:
LOG_FILE_PATH = '/content/drive/MyDrive/Colab Notebooks/Neoversity/DAA/HW5/lms-stage-access.log'

In [ ]:
def load_ips(filepath):
    """Читає файл і витягує remote_addr"""
    ips = []
    print(f"Зчитування файлу: {filepath} ...")
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            for line in f:
                try:
                    entry = json.loads(line)
                    if 'remote_addr' in entry:
                        ips.append(entry['remote_addr'])
                except (json.JSONDecodeError, ValueError):
                    continue
    except FileNotFoundError:
        print(f"Файл за шляхом '{filepath}' не знайдено.")
        return []

    print(f"Завантажено {len(ips)} рядків.")
    return ips

In [ ]:
def count_exact(ips):
    """Рахує кількість унікальних IP-адрес методом set()."""
    t_start = time.time()
    unique_count = len(set(ips))
    t_end = time.time()
    return unique_count, t_end - t_start

In [ ]:
def count_hll(ips):
    """Рахує кількість унікальних IP-адрес методом HyperLogLog."""
    t_start = time.time()
    hll = HyperLogLog(p=14)
    for ip in ips:
        hll.update(ip.encode('utf-8'))
    count = hll.count()
    t_end = time.time()
    return count, t_end - t_start

In [ ]:
if __name__ == "__main__":
    # Завантаження
    ip_data = load_ips(LOG_FILE_PATH)

    if ip_data:
        # Підрахунки
        exact_count, exact_time = count_exact(ip_data)
        hll_count, hll_time = count_hll(ip_data)

        # Виведення таблиці
        print("\nРезультати порівняння:")
        print(f"{'':<25} {'Точний підрахунок':<20} {'HyperLogLog':<20}")
        print(f"{'Унікальні елементи':<25} {exact_count:<20} {hll_count:<20}")
        print(f"{'Час виконання (сек.)':<25} {exact_time:<20.5f} {hll_time:<20.5f}")

        err = abs(exact_count - hll_count) / exact_count * 100
        print(f"\nПохибка HLL: {err:.2f}%")

Зчитування файлу: /content/drive/MyDrive/Colab Notebooks/Neoversity/DAA/HW5/lms-stage-access.log ...
Завантажено 29553 рядків.

Результати порівняння:
                          Точний підрахунок    HyperLogLog         
Унікальні елементи        28                   28.023953075428718  
Час виконання (сек.)      0.00151              0.06812             

Похибка HLL: 0.09%


Як бачимо HLL дуже непогано впорався  - 28 проти 28,02 від апроксимованого методу при цьому час у HLL вийшов в 40 разів більший.

Це можна пояснити тим, що метод set() оптимізований на С. HLL же, в свою чергу, навпаки застосовую хешування через Python для кожного елементу, що дає збільшення часу виконання і процесорне навантаження значно більше ніж в методі set(). При цьому при використанні HLL зберігаються лише 28 унікальних хеш в пам'яті для аналізу 29553 елементів данного лога, і в цьому головна перевага метода - він дозволяє доволі точно порахувати кардинальність виборки данних при мінімальному використанні пам'яті. Довше, але система не ляже.